### Generate text description

In [13]:
import pandas as pd
import numpy as np
from utils.openai import client
from transformers import AutoTokenizer


In [ ]:
path = "/home/wenhao/Project/greatxue/polyTexter/proj/datasets/raw_comprehensive_tg.csv"
df_raw = pd.read_csv(path)
df = df_raw[['SMILES', 'tg']]
df

,SMILES,tg
0,*C1COC2C1OCC2Oc1ccc(cc1)CNC(=O)CCCCCCC(=O)NCc1...,21.581731
1,*OC(CCC(OC(=O)Nc1ccc(cc1)Cc1ccc(cc1)NC(=O)*)C)C,63.589338
2,*OC(=O)c1ccc(cc1)C(=O)OCCCC(=O)NCc1ccc(cc1)CNC...,53.557261
3,*OC(=O)NCCNC(=O)OCC*,5.896093
4,*SCCCCC*,-55.378610
...,...,...
561,*NNC(=O)CCC(=O)NNC(=O)CCCCCCCCC(=O)*,64.698504
562,*NC(=O)CCC(=O)NCCCCCCCC*,69.221302
563,*NC(C(C(=O)*)(C)C)c1ccccc1,154.359507
564,*OC(=O)C/C=C/CC(=O)OCCCCCCCCCCCCCC*,-41.101589


In [3]:
def ques_gpt(prompt="Hi, how do you do today?"):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
    )
    if response.choices[0]:
        return response.choices[0].message.content.strip()
    else:
        print('No valid GPT-response generated.')

In [ ]:
responses = []

for index, row in df.iterrows():
    smiles = row['SMILES']
    print(f"{index} Processing SMILES: {smiles}")
    gpt_response = ques_gpt(f"From a structural perspective, please provide some inferential information related to the glass transition temperature of the polymer represented by the SMILES {smiles}.")
    #print(f"{index} GPT Response: {gpt_response}\n")
    responses.append(gpt_response)

prompt_df = pd.DataFrame(responses, columns=['tg_description'])
prompt_df.to_csv("/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_comprehensive_description.csv", index=True)
print("The description has been saved.")


0 Processing SMILES: *C1COC2C1OCC2Oc1ccc(cc1)CNC(=O)CCCCCCC(=O)NCc1ccc(cc1)O*


1 Processing SMILES: *OC(CCC(OC(=O)Nc1ccc(cc1)Cc1ccc(cc1)NC(=O)*)C)C
2 Processing SMILES: *OC(=O)c1ccc(cc1)C(=O)OCCCC(=O)NCc1ccc(cc1)CNC(=O)CCC*
3 Processing SMILES: *OC(=O)NCCNC(=O)OCC*
4 Processing SMILES: *SCCCCC*
5 Processing SMILES: *Oc1ccc(cc1)C(=O)OC(=O)c1ccc(cc1)OCCCCCC*
6 Processing SMILES: *c1[nH]c(cc1CC(=O)OCCCCCCCC)*
7 Processing SMILES: *C(C*)(CC(=O)OCCCCCCCCCC)C(=O)OCCCCCCCCCC
8 Processing SMILES: *OCC1C(C1)C*
9 Processing SMILES: *N(C(=O)CCCCCCCCCCCCCCCCC(=O)N(CCCCCC*)C)C
10 Processing SMILES: *O[Si](*)(CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)C
11 Processing SMILES: *S(=O)(=O)c1ccc(cc1)C(=O)CNc1ccc(cc1)NCC(=O)c1ccc(cc1)*
12 Processing SMILES: *S(=O)(=O)c1ccc(cc1)C(=O)NCCNC(=O)c1ccc(cc1)*
13 Processing SMILES: *c1cc2c(C(=O)N(C2=O)c2ccc(cc2)C(c2ccc(cc2)N2C(=O)c3c(C2=O)cc(cc3)C(=O)*)CCCCC)cc1
14 Processing SMILES: *NC(=O)C(=O)NCCCCCCCCCCNC(=O)CCCCCCCCC(=O)NCCCCCCCCCC*
15 Processing SMILES: *OC(=O)c1ccc(cc1)C(=O)OCCCCCCCCCCCCCCCCCCCC*
16 Processing SMILES: *Oc1ccc(cc1)

In [ ]:
path = "/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_comprehensive_description.csv"
df_text = pd.read_csv(path)

df_text

,Unnamed: 0,tg_description
0,0,To infer the glass transition temperature (\( ...
1,1,The SMILES string provided represents a polyme...
2,2,To infer the glass transition temperature (\( ...
3,3,The polymer represented by the SMILES *OC(=O)N...
4,4,The polymer represented by the SMILES *SCCCCC*...
...,...,...
561,561,The SMILES string you provided describes a pol...
562,562,To infer the glass transition temperature (Tg)...
563,563,"The provided SMILES notation, *NC(C(C(=O)*)(C)..."
564,564,The given SMILES notation represents a polymer...


### Generate tokenized descriptions

In [5]:
df_merged = pd.concat([df, df_text[['tg_description']]], axis=1)
df_merged.to_csv('smiles_with_description_tg.csv', index=True)

In [7]:
########################## For Test ############################
model_name = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

sample_text = "This is tokenizer."
tokens = tokenizer.encode(sample_text, add_special_tokens=True)

print("Original text:", sample_text)
print("Tokens:", tokens)
print("Decoded text:", tokenizer.decode(tokens))

Original text: This is tokenizer.
Tokens: [128000, 2028, 374, 47058, 13]
Decoded text: <|begin_of_text|>This is tokenizer.


In [16]:
model_name = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

input_csv_path = '/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_comprehensive_description.csv'
df = pd.read_csv(input_csv_path)

if 'tg_description' not in df.columns:
    raise ValueError("输入 CSV 中必须包含 'tg_description' 列")

tokens_list = []

for description in df['tg_description']:
    tokens = tokenizer.encode(description, add_special_tokens=True)
    tokens_list.append(tokens)

print(tokens_list)
'''
array = np.array(tokens_list)
output_npy_path = 'tokenized_description.npy'
np.save(output_npy_path, array)
print("===================The npy file is saved.===================")
'''
df = pd.DataFrame(tokens_list)
output_csv_path = 'tokenized_description.csv'
df.to_csv(output_csv_path, index=False, header=False) 
print("===================The csv file is saved.===================")

[[128000, 1271, 24499, 279, 9168, 9320, 9499, 20374, 7, 350, 1928, 1144, 595, 315, 279, 47393, 15609, 555, 279, 14031, 47078, 353, 34, 16, 34, 7767, 17, 34, 16, 46, 3791, 17, 46, 66, 16, 38154, 32524, 16, 8, 34, 10153, 121110, 46, 8, 49032, 54973, 121110, 46, 8, 10153, 66, 16, 38154, 32524, 16, 8, 46, 12594, 584, 649, 21635, 1202, 24693, 6956, 323, 4519, 13, 4815, 16, 13, 3146, 39264, 1195, 56527, 13552, 19025, 334, 512, 256, 482, 578, 47393, 56527, 5727, 77102, 14726, 11, 2737, 264, 294, 24620, 2194, 10264, 4712, 34, 16, 34, 7767, 17, 34, 16, 46, 3791, 17, 3849, 323, 82688, 25562, 4712, 66, 16, 38154, 32524, 16, 4911, 323, 353, 66, 16, 38154, 32524, 16, 8, 46, 9, 570, 720, 256, 482, 4314, 25562, 17210, 311, 24693, 13552, 19025, 11, 902, 8965, 12992, 18240, 350, 1928, 1144, 3677, 17, 13, 3146, 3386, 76, 43943, 22639, 334, 512, 256, 482, 40602, 4469, 64186, 25, 578, 9546, 315, 17055, 87, 4010, 5315, 4712, 66, 16, 38154, 32524, 16, 8, 46, 3849, 323, 1097, 579, 5315, 4712, 34, 10153, 1211